# 🗑️ Waste Classification using YOLOv8
## TASK 2: Thiết lập môi trường và làm quen YOLOv8

**Project:** Phân loại rác thải thông minh cho dây chuyền xử lý công nghiệp  
**Classes:** Plastic, Metal, Paper, Glass  
**Model:** YOLOv8n (nano)  
**Environment:** Google Colab with GPU

---


## 📦 CELL 1: Install YOLOv8 và Check GPU

**Mục đích:**
- Cài đặt thư viện Ultralytics YOLOv8
- Import các libraries cần thiết
- Kiểm tra GPU có available không

**⚠️ Lưu ý:** Đảm bảo đã enable GPU: `Runtime > Change runtime type > T4 GPU`


In [ ]:
# Install Ultralytics YOLOv8
print("📦 Installing YOLOv8...")
%pip install ultralytics -q

# Import libraries
from ultralytics import YOLO
import torch
from google.colab import drive
from IPython.display import Image, display
import os
import glob
import shutil

print("\n✅ Installation completed!")

# Check GPU (QUAN TRỌNG!)
print("\n🔍 GPU Information:")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"\n🎉 GPU is ready for training!")
else:
    print("\n⚠️ WARNING: No GPU found!")
    print("Please enable GPU: Runtime > Change runtime type > T4 GPU")
    print("Then restart this cell.")


## 📂 CELL 2: Mount Google Drive

**Mục đích:**
- Kết nối Google Drive để access dataset
- Verify folder structure

**⚠️ Lưu ý:** Khi chạy cell này, sẽ có popup yêu cầu authorization


In [ ]:
# Mount Google Drive
print("📂 Mounting Google Drive...")
drive.mount('/content/drive')

# Verify mount
if os.path.exists('/content/drive/MyDrive'):
    print("\n✅ Google Drive mounted successfully!")
    
    # List files in project folder
    project_folder = '/content/drive/MyDrive/Waste_Detection_Project'
    if os.path.exists(project_folder):
        print(f"\n📁 Files in project folder:")
        !ls -lh /content/drive/MyDrive/Waste_Detection_Project/
    else:
        print(f"\n⚠️ Folder not found: {project_folder}")
        print("Please check the folder name!")
else:
    print("\n❌ Failed to mount Google Drive!")


## 📦 CELL 3: Unzip Dataset

**Mục đích:**
- Giải nén dataset từ Drive vào Colab
- Verify cấu trúc dataset
- Đếm số lượng images/labels

**⚠️ Lưu ý:** Sửa tên file zip nếu bạn đặt tên khác


In [ ]:
# Unzip dataset
print("📦 Extracting dataset...")

# Path to zip file (SỬA TÊN NẾU CẦN!)
zip_path = '/content/drive/MyDrive/Waste_Detection_Project/waste_dataset.zip'

if not os.path.exists(zip_path):
    print(f"❌ File not found: {zip_path}")
    print("\nAvailable files:")
    !ls -la /content/drive/MyDrive/Waste_Detection_Project/
else:
    !unzip -q {zip_path} -d /content/
    print("✅ Dataset extracted!")
    
    print("\n📊 Dataset structure:")
    !ls -la /content/dataset/
    
    # Count files
    train_imgs = len(glob.glob('/content/dataset/train/images/*'))
    train_labels = len(glob.glob('/content/dataset/train/labels/*'))
    valid_imgs = len(glob.glob('/content/dataset/valid/images/*'))
    valid_labels = len(glob.glob('/content/dataset/valid/labels/*'))
    test_imgs = len(glob.glob('/content/dataset/test/images/*'))
    test_labels = len(glob.glob('/content/dataset/test/labels/*'))
    
    print(f"\n📁 File counts:")
    print(f"  Train: {train_imgs} images, {train_labels} labels")
    print(f"  Valid: {valid_imgs} images, {valid_labels} labels")
    print(f"  Test:  {test_imgs} images, {test_labels} labels")
    print(f"  Total: {train_imgs + valid_imgs + test_imgs} images")
    
    if train_imgs == train_labels and valid_imgs == valid_labels:
        print("\n✅ Images and labels match!")


## 🔧 CELL 4: Fix data.yaml Paths

**Mục đích:**
- Update paths trong data.yaml cho Colab
- Chuyển từ relative paths sang absolute paths


In [ ]:
# Fix data.yaml paths for Colab
data_yaml_path = '/content/dataset/data.yaml'

yaml_content = """# Dataset configuration for YOLOv8
train: /content/dataset/train/images
val: /content/dataset/valid/images
test: /content/dataset/test/images

nc: 4
names: ['Plastic', 'Metal', 'Paper', 'Glass']
"""

with open(data_yaml_path, 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml updated for Colab!")
print("\nContent:")
print(yaml_content)


## 🧪 CELL 5: Test YOLOv8 Pretrained Model

**Mục đích:**
- Test YOLOv8 với pretrained model
- Verify setup hoạt động OK

**⚠️ Lưu ý:** Predictions sẽ SAI vì model chưa train cho waste!


In [ ]:
# Test pretrained YOLOv8
print("🤖 Loading pretrained YOLOv8n...")
model = YOLO('yolov8n.pt')

print("✅ Model loaded!\n")

# Test on sample images
sample_images = sorted(glob.glob('/content/dataset/train/images/*'))[:3]

print(f"Testing on {len(sample_images)} images...")

results = model.predict(
    source=sample_images,
    imgsz=640,
    conf=0.25,
    save=True,
    project='/content/test_inference',
    name='pretrained_test',
    exist_ok=True
)

print("\n✅ Inference completed!")
print("⚠️ Classes will be WRONG (trained on COCO, not waste)")

# Display results
pred_images = sorted(glob.glob('/content/test_inference/pretrained_test/*.jpg'))
for i, img_path in enumerate(pred_images[:3], 1):
    print(f"\n--- Image {i} ---")
    display(Image(img_path, width=600))


---

## ✅ TASK 2 COMPLETED!

### Checklist:
- ✅ YOLOv8 installed
- ✅ GPU enabled and verified
- ✅ Google Drive mounted
- ✅ Dataset extracted (~1981 images)
- ✅ data.yaml paths updated
- ✅ Pretrained model tested

### 🎯 Next: TASK 3 - TRAINING

**Ready to start training!**

Estimated time: 2-4 hours (100 epochs)

---

**📝 Notes:**
- Backup weights to Drive during training
- Monitor progress regularly  
- Colab may disconnect after 12h

**🚀 Good luck with training!**
